# DATA SPLIT - testing the full approach - COGNITION
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## define a fixed samplerate

In [2]:
samplerate=200

## load mass

In [4]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [6]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [7]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## create data split
LOOCV with 3 subjects for validation by fold

In [8]:
#define subjects used in the experiment, keep only annotated by two experts
usedSubjects=np.array(signalsMetadata['subjectId'])
usedSubjects

array(['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008',
       '0009'], dtype=object)

In [9]:
valCount=2  #number of signals for validation (selected randomly)

trainSplits=[]
valSplits=[]
testSplits=[]
for i in range(len(usedSubjects)):
    thisTest=usedSubjects[i]
    thisNoTest=np.setdiff1d(usedSubjects,thisTest)
    thisVal=np.random.choice(thisNoTest,size=valCount,replace=False)
    thisTrain=np.setdiff1d(thisNoTest,thisVal)
    trainSplits.append(thisTrain)
    valSplits.append(thisVal)
    testSplits.append(thisTest)

dataSplits=pd.DataFrame({
    'train':trainSplits,
    'val':valSplits,
    'test':testSplits
})

dataSplits

,train,val,test
0,"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001
1,"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002
2,"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003
3,"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004
4,"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005
5,"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006
6,"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007
7,"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008
8,"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009


In [10]:
dumpPickle("dataSplits_LOOCV.pkl",dataSplits)